In [1]:
# Import libraries: to Ingest data
import package
import mysql.connector
import csv
import pandas as pd
from pandas import DataFrame
from mysql.connector import errorcode

### Step 1. Install MySQL Python connector
* mysql-connector-python is a MySQL database adapter in Python. It provides convenient APIs to load and query the tables. It also has a nice tool to load CSV files into the tables. In this step, we will need to install this Python module.

~~~
pip3 install mysql-connector-python
~~~

### Step 2. Load third-party ticket sales data into MySQL database
> __Step 2.1 Setup database connection__

> * In order to make a query against the database table, we need to first connect to it. A connection can be established only when the user provides the proper target host, port, and user credentials.

In [2]:
# Setup database connection and connect to MySQL database
def get_db_connection():
    connection = None
    try:
        connection = mysql.connector.connect(user='xxxxxxx',
        password='xxxxxxxx',
        host='127.0.0.1',
        port='3306',
        database='mini_pipeline')
    except Exception as error:
        print("Error while connecting to database for job tracker", error)
    return connection

> __Step 2.2 Load CSV to table__

> * You’ll find the third party vendor data in the CSV file provided to you. 
> * The CSV follows the schema of the table. You will need to use the Python connector to insert each record of the CSV file into the “sales” table.

In [3]:
# Extract data
## Use package.sales to pull data from relative file location
sales = package.sales
file_path_csv = r'data\sales_clean.csv'

# Tranform data
## Append lables to dataframe
df = DataFrame(sales,columns=['ticket_id', 'trans_date', 'event_id', 'event_name', 'event_date', 
                              'event_type', 'event_city', 'customer_id', 'price', 'num_tickets'
                             ])

## Convert columns to the appropriate data types
df1 = df.astype({'ticket_id': 'int', 'trans_date': 'datetime64[ns]', 'event_id': 'int', 'event_name': 'str', 'event_date': 'datetime64[ns]', 
                 'event_type': 'str', 'event_city': 'str', 'customer_id': 'int', 'price': 'float', 'num_tickets': 'int'
                })

## Save .csv file
df1.to_csv(file_path_csv, index=False)

## Notes
#print(file_path_csv) # print string (file name)
#print(open(file_path_csv)) # print file object
#print(open(file_path_csv).read()) # printed file content
#print(open(file_path_csv).readlines()) # printed file content

In [4]:
## Create table and Load data to database
def load_third_party(connection, file_path_csv):
    with connection.cursor() as cursor:
        cursor = connection.cursor()

        # [Iterate through the CSV file and execute insert statement]
        # Check database and drop target table if it exists
        cursor.execute("DROP TABLE IF EXISTS sales")
        
        # Create sale table SCHEMA
        create_sales_table_query = """
        CREATE TABLE sales(
            ticket_id INT,
            trans_date DATE,
            event_id INT,
            event_name VARCHAR(50),
            event_date DATE NOT NULL,
            event_type VARCHAR(10),
            event_city VARCHAR(20),
            customer_id INT,
            price DECIMAL,
            num_tickets INT,
            PRIMARY KEY(ticket_id)
        )
        """
        # Execute the table query
        cursor.execute(create_sales_table_query)

        # Open csv file object (container)
        csv_data = csv.reader(open(file_path_csv, 'r'))
        ctr = 0 # set interable constructor at 0
        for row in csv_data: # iterate over rows in csv data
            if ctr == 0:
                ctr += 1
                continue
            # Print test message to ensure successful load
            print(f"Successful Load: {row}.")
            
            # Execute INSERT statement
            cursor.execute('INSERT INTO mini_pipeline.sales (ticket_id, trans_date, event_id, event_name, event_date, event_type, event_city, customer_id, price, num_tickets)' \
                  'VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)', 
                  row)
    # Commit the connection to the database.
    connection.commit()
 
load_third_party(get_db_connection(), file_path_csv)

Successful Load: ['1', '2020-08-01', '100', 'The North American International Auto Show', '2020-09-01', 'Exhibition', 'Michigan', '123', '35.0', '3'].
Successful Load: ['2', '2020-08-03', '101', 'Carlisle Ford Nationals', '2020-09-30', 'Exhibition', 'Carlisle', '151', '43.0', '1'].
Successful Load: ['3', '2020-08-03', '102', 'Washington Spirits vs Sky Blue FC', '2020-08-30', 'Sports', 'Washington DC', '223', '59.34', '5'].
Successful Load: ['4', '2020-08-05', '103', 'Christmas Spectacular', '2020-10-05', 'Theater', 'New York', '223', '89.95', '2'].
Successful Load: ['5', '2020-08-05', '100', 'The North American International Auto Show', '2020-09-01', 'Exhibition', 'Michigan', '126', '35.0', '1'].
Successful Load: ['6', '2020-08-05', '103', 'Christmas Spectacular', '2020-10-05', 'Theater', 'New York', '1024', '89.95', '3'].


### Step 3. Display statistical information
* After the data is loaded into the table, you can use this data to provide recommendations to the user. For instance, recommending popular events by finding the most top-selling tickets for the past month.

> __3.1 Query the table and get the selected records__

In [5]:
# Build popular query function
def query_popular_tickets1(connection):
    # Get the most popular ticket in the past month
    sql_statement = "SELECT event_name, event_type, event_city, MAX(num_tickets) FROM mini_pipeline.sales as mps"
    cursor = connection.cursor()
    cursor.execute(sql_statement)
    records = cursor.fetchall()
    cursor.close()
    return records

# Run query function
trending_event = query_popular_tickets1(get_db_connection())
print(f'The most trending event at the moment is: {trending_event}')

The most trending event at the moment is: [('The North American International Auto Show', 'Exhibition', 'Michigan', 5)]


> __3.2 Display the result__
> * The records you just retrieved are formatted as a list of tuples. You need to convert the format to display the on-screen results in a more user-friendly format. Please use this as an example.

> ___Here are the most popular tickets in the past month:___
> - The North American International Auto Show
> - Carlisle Ford Nationals
> - Washington Spirits vs Sky Blue FC

In [6]:
# Build popular query function
fileout_csv = r'data\fileout.csv'

def query_popular_tickets2(connection):
    # Get the most popular ticket in the past month
    sql_statement = "SELECT event_name FROM mini_pipeline.sales as mps LIMIT 3"
    cursor = connection.cursor()
    cursor.execute(sql_statement)
    with open(fileout_csv,'w') as fileout:
        writer = csv.writer(fileout)
        for row in cursor:
            writer.writerow(row)
    dframe = pd.read_table(fileout_csv, sep=',')
    print(dframe.to_string(index = False))
    
    # Close the connection to the database.
    connection.close()

# Run query function
query_popular_tickets2(get_db_connection())

The North American International Auto Show
                   Carlisle Ford Nationals
         Washington Spirits vs Sky Blue FC
